# GCN for DAVIS 2016

In this notebook, a custom [PyTorch Geometric](https://rusty1s.github.io/pytorch_geometric/build/html/index.html) [InMemoryDataset](https://rusty1s.github.io/pytorch_geometric/build/html/_modules/torch_geometric/data/in_memory_dataset.html#InMemoryDataset) for the DAVIS 2016 dataset is created. The implementation is based on this [tutorial](https://rusty1s.github.io/pytorch_geometric/build/html/notes/create_dataset.html). The dataset is then used to train a simple GCN network as a first evaluation based on this [tutorial](https://rusty1s.github.io/pytorch_geometric/build/html/notes/introduction.html#learning-methods-on-graphs).

The dataset consists of single PyTorch Geometric [Data](https://rusty1s.github.io/pytorch_geometric/build/html/_modules/torch_geometric/data/data.html#Data) objects which model a single graph with various attributes. For this dataset, a graph for each contour is created. Hereby, each node of the graph represents one contour point. The feature of each node is the OSVOS feature vector from the next frame at this point. Each node is connected to its K nearest neighbours. The feature of each edge is the distance between the nodes it connects. The targets of each node is the translation it undergoes from the current to the next frame.

## Imports

In [1]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

from pg_datasets.davis_2016 import DAVIS2016

## Paths & Constants

In [2]:
CONTOURS_FOLDERS_PATH = 'DAVIS_2016/DAVIS/Contours/480p'
TRANSLATIONS_FOLDERS_PATH = 'DAVIS_2016/DAVIS/Translations/480p'
PYTORCH_GEOMETRIC_DAVIS_2016_DATASET_PATH = 'pg_datasets/DAVIS_2016'

SKIP_SEQUENCES = ['bmx-trees', 'bus', 'cows', 'dog-agility', 'horsejump-high', 
                  'horsejump-low', 'kite-walk', 'lucia', 'libby', 'motorbike',
                  'paragliding', 'rhino', 'scooter-gray', 'swing']
K = 32

## Dataset

In [3]:
dataset = DAVIS2016(PYTORCH_GEOMETRIC_DAVIS_2016_DATASET_PATH, CONTOURS_FOLDERS_PATH,
                    TRANSLATIONS_FOLDERS_PATH, K, SKIP_SEQUENCES)

Processing...
#1: drift-chicane
#3: soapbox
#4: breakdance
#5: drift-turn
#6: mallard-fly
#9: scooter-black
#10: breakdance-flare
#12: elephant
#14: rollerblade
#15: dance-twirl
#16: dance-jump
#18: mallard-water
#19: car-turn
#22: car-shadow
#23: paragliding-launch
#24: stroller
#25: bear
#26: hockey
#27: dog
#28: boat
#29: car-roundabout
#30: soccerball
#31: train
#32: tennis
#33: parkour
#34: surf
#35: kite-surf
#37: drift-straight
#38: flamingo
#39: goat
#41: hike
#42: motocross-jump
#44: motocross-bumps
#47: camel
#48: blackswan
#49: bmx-bumps
Done!


## Simple GCN

In [4]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(dataset.num_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

AttributeError: 'NoneType' object has no attribute 'dim'

In [ ]:
model.eval()
_, pred = model(data).max(dim=1)
correct = float (pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
acc = correct / data.test_mask.sum().item()
print('Accuracy: {:.4f}'.format(acc))